<a href="https://colab.research.google.com/github/naveedkhalid091/Learn_Agentic_AI/blob/main/step02_generative_ai_for_beginners/01_projects/02_RAG_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
# libraries/SDK & vector database installation

!pip install -qU langchain-google-genai langchain-pinecone

In [44]:
# Initialization of pinecone

from pinecone import Pinecone, ServerlessSpec

from google.colab import userdata
pinecone_api_key= userdata.get('PINECONE_API_KEY')

pc=Pinecone(api_key=pinecone_api_key)

In [ ]:
# Index creating
index_name = "second-rag-project"  # change if desired

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
)

index = pc.Index(index_name)

In [58]:
## Embedding model selection

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [59]:
## Adding Embeddings into a wrapper

from langchain_pinecone import PineconeVectorStore

vector_store= PineconeVectorStore(index=index, embedding=embeddings)

In [48]:
from google.colab import files
uploaded = files.upload()

In [60]:
# install pypdf
!pip install langchain pypdf


In [61]:


!pip install -U langchain-community


In [62]:
# Load and process the pdf file.
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader('/content/Introduction to myself.pdf')

documents = loader.load()

In [63]:
import os

from google.colab import userdata
GOOGLE_API_KEY='GOOGLE_API_KEY'
os.environ[GOOGLE_API_KEY]=userdata.get('GOOGLE_API_KEY')

In [64]:
from langchain_pinecone import PineconeVectorStore

vector_store= PineconeVectorStore(index=index, embedding=embeddings)

In [65]:
from uuid import uuid4

from langchain_core.documents import Document

document_1=Document(
    page_content="Introduction to myself",
    metadata={"/content/Introduction to myself.pdf":"introduction"},
)

In [66]:
len(documents)

1

In [67]:
from uuid import uuid4
uuid4()  # Generates Random id number every time


UUID('091d75d9-c880-42a9-b3f6-fa56dd2cf03d')

In [68]:

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['3b85e977-e48a-4265-84ed-1ae596e7cfd6']

In [69]:
results = vector_store.similarity_search(
    "Naveed",
    k=2,
    filter={"/content/Introduction to myself.pdf":"introduction"}
    )

for res in results:
    print(f'''{res.page_content} [{res.metadata}]''')

In [70]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [71]:
# define a function
def answer_to_user(query:str):
  # Vector Search
  vector_results = vector_store.similarity_search(query, k=1)
  # Passing to model Vector Results + User Query
  final_answer = llm.invoke (f'''Answer this user Query: {query}, Here are some ref to answer {vector_results}, ''')
  return final_answer

In [82]:
answer=answer_to_user("How many years of experience Naveed has ?")

In [83]:
answer.content

'Naveed has 10 years of industry experience from UK-based companies.\n'